Dataset link - https://github.com/florex/resume_corpus

In [1]:
import os
import pandas as pd
import numpy as np
import unicodedata
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
file_path = r'C:\Coding\Jupyter notebooks\resume_corpus-master\resume_samples\resume_samples.txt'

with open(file_path, 'r', encoding='latin1') as file:
    lines = file.readlines()

# Initialize empty lists to store data and line numbers where the if condition is not met
ids = []
occupations = []
resumes = []
problematic_lines = []

# Process each line and extract the data
for line_number, line in enumerate(lines, 1):  # Start line_number from 1
    parts = line.strip().split(':::')
    if len(parts) == 3:
        ids.append(parts[0])
        occupations.append(parts[1])
        resumes.append(parts[2])
    else:
        problematic_lines.append(line_number)

# Create the DataFrame
resume_data = pd.DataFrame({'ID': ids, 'Occupations': occupations, 'Resume': resumes})

# Print the line numbers where the if condition is not met
print("Problematic lines:", problematic_lines)

# Convert the line numbers to list indices (subtract 1)
problematic_indices = [line_number - 1 for line_number in problematic_lines]

Problematic lines: [4278, 21416, 25427]


In [3]:
resume_data.head()

,ID,Occupations,Resume
0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator;Database Administrator;...,"Database Administrator <span class=""hl"">Databa..."
1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,"Database Administrator;SQL, Microsoft PowerPoi...","Database Administrator <span class=""hl"">Databa..."
2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Oracle Database Administrator;Oracle Database ...,Oracle Database Administrator Oracle <span cla...
3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Amazon Redshift Administrator and ETL Develope...,Amazon Redshift Administrator and ETL Develope...
4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Scrum Master;Oracle Database Administrator/ Sc...,Scrum Master Scrum Master Scrum Master Richmon...


In [4]:
# Directory path of the resumes_corpus folder containing .lab files
lab_folder_path = 'C:/Coding/Jupyter notebooks/resume_corpus-master/resumes_corpus/'

# List all the .lab files in the resumes_corpus folder
lab_files = [file for file in os.listdir(lab_folder_path) if file.endswith('.lab')]

# Initialize an empty list to store the extracted labels
labels_list = []

# Read each .lab file and extract the labels
for lab_file in lab_files:
    with open(os.path.join(lab_folder_path, lab_file), 'r') as file:
        labels = file.read().strip()
        labels_list.append(labels)

# Remove the labels for problematic resume data using the indices
labels_list = [labels_list[i] for i in range(len(labels_list)) if i not in problematic_indices]        

# Merge the extracted labels with the existing dataframe
resume_data['Labels'] = labels_list

resume_data.head()

,ID,Occupations,Resume,Labels
0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator;Database Administrator;...,"Database Administrator <span class=""hl"">Databa...",Database_Administrator
1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,"Database Administrator;SQL, Microsoft PowerPoi...","Database Administrator <span class=""hl"">Databa...",Database_Administrator
2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Oracle Database Administrator;Oracle Database ...,Oracle Database Administrator Oracle <span cla...,Database_Administrator
3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Amazon Redshift Administrator and ETL Develope...,Amazon Redshift Administrator and ETL Develope...,Database_Administrator
4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Scrum Master;Oracle Database Administrator/ Sc...,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator


In [5]:
def preprocess_normalized_classes(file_path):
    normalized_mapping = {}
    with open(file_path, 'r', encoding='latin-1') as file:
        for line in file:
            line = line.strip()
            # Use regular expressions to find original occupations and normalized occupation
            #regex is used because the data is inconsistently seperated by ":" and " : "
            match = re.match(r'(.+?)\s*:\s*(.+)', line)
            if match:
                original_occupations, normalized_occupation = match.groups()
                #this is done to handle inconsistency in use seperators for original occupation like "/", "\", "\x95" 
                original_occupations = [occ.strip().replace('\x95', '').lower() for occ in re.split(r'[\\\/\x95]', original_occupations)]
                normalized_occupation = normalized_occupation.strip()

                for original_occ in original_occupations:
                    normalized_mapping[original_occ] = normalized_occupation
                    
    #df = pd.DataFrame(list(normalized_mapping.items()), columns=['Original_Occupation', 'Normalized_Occupation'])
    return normalized_mapping


normal_path = 'C:/Coding/Jupyter notebooks/resume_corpus-master/normlized_classes.txt'
normalised_mapping = preprocess_normalized_classes(normal_path)
normalised_mapping

{'front end developer (branding specialist)': 'Front_End_Developer',
 'mis network analyst': 'Network_Administrator',
 'designer & front- end developer': 'Front_End_Developer',
 'network administrator politécnico 5 septiembre': 'Network_Administrator',
 'web developer': 'Database_Administrator,Web_Developer',
 '.net developer': 'Web_Developer',
 'senior project manager - united launch alliance (ula)': 'Project_manager',
 'product manager': 'Systems_Administrator',
 'project manager': 'Project_manager',
 'network': 'Systems_Administrator,Network_Administrator',
 'firewall engineer': 'Network_Administrator,Security_Analyst',
 'web producer': 'Web_Developer,Front_End_Developer',
 'database specialist & compliancewire administrator': 'Database_Administrator',
 'manager of it infrastructure and operations': 'Project_manager',
 'security analyst': 'Security_Analyst',
 'software engineer': 'Software_Developer',
 'systems analyst': 'Project_manager',
 'oracle production dba': 'Database_Adminis

In [6]:
def normalize_occupations(row, normalized_mapping):
    occupations = row['Occupations'].lower().split(';')
    normalized_occupations_set = set()  # Use a set to store unique normalized occupations

    for occ in occupations:
        normalized_occ = normalized_mapping.get(occ.strip(), None)
        if normalized_occ:
            normalized_occupations_set.add(normalized_occ)

    normalized_occupations = list(normalized_occupations_set)
    return ';'.join(normalized_occupations)

In [7]:
resume_data['Normalized_Occupation'] = resume_data.apply(normalize_occupations, args=(normalised_mapping,), axis=1)
resume_data.head()

,ID,Occupations,Resume,Labels,Normalized_Occupation
0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator;Database Administrator;...,"Database Administrator <span class=""hl"">Databa...",Database_Administrator,"Front_End_Developer;Database_Administrator,Sys..."
1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,"Database Administrator;SQL, Microsoft PowerPoi...","Database Administrator <span class=""hl"">Databa...",Database_Administrator,"Database_Administrator,Web_Developer"
2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Oracle Database Administrator;Oracle Database ...,Oracle Database Administrator Oracle <span cla...,Database_Administrator,Database_Administrator;Front_End_Developer;Sys...
3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Amazon Redshift Administrator and ETL Develope...,Amazon Redshift Administrator and ETL Develope...,Database_Administrator,"Database_Administrator;Database_Administrator,..."
4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Scrum Master;Oracle Database Administrator/ Sc...,Scrum Master Scrum Master Scrum Master Richmon...,Database_Administrator,"Web_Developer,Project_manager;Database_Adminis..."


In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rajat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
lemmatizer = WordNetLemmatizer()

def preprocess_resume_text(text):
    # Remove HTML tags and special characters using regular expressions
    cleaned_text = re.sub(r'<[^>]+>', ' ', text)
    cleaned_text = re.sub(r'[^\w.+/#-]', ' ', cleaned_text)

    # Convert to lowercase
    cleaned_text = cleaned_text.lower()

    #Tokenize text into words
    words = word_tokenize(cleaned_text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # Join words back into a single string
    preprocessed_text = ' '.join(filtered_words)

    return preprocessed_text

In [10]:
resume_data['Preprocessed_Resume'] = resume_data['Resume'].apply(preprocess_resume_text)

In [11]:
output_file_path = 'C:/Coding/Jupyter notebooks/resume_data1.csv'

# Save the 'resume_data' DataFrame as a CSV file
resume_data.to_csv(output_file_path, index=False)

print("DataFrame saved as CSV file successfully.")

DataFrame saved as CSV file successfully.


In [11]:
norm_labels = [labels.split(';') for labels in resume_data['Normalized_Occupation']]
unique_classes = set()

# Loop through each row in y_train_labels and split the labels
for row in norm_labels:
    unique_classes.update(row)

# Convert the set back to a list for indexing
class_list = list(unique_classes)
class_list

['',
 'Software_Developer,Systems_Administrator,Project_manager',
 'Database_Administrator,Python_Developer',
 'Database_Administrator,Front_End_Developer',
 'Java_Developer,Software_Developer',
 'Java_Developer,Front_End_Developer',
 'Python_Developer,Network_Administrator',
 'Database_Administrator,Software_Developer,Systems_Administrator',
 'Web_Developer,Software_Developer,Systems_Administrator',
 'Web_Developer,Systems_Administrator',
 'Web_Developer,Python_Developer,Software_Developer',
 'Database_Administrator,Java_Developer,Python_Developer,Software_Developer',
 'Systems_Administrator,Project_manager',
 'Software_Developer,Systems_Administrator',
 'Systems_Administrator,Project_manager,Security_Analyst',
 'Java_Developer,Python_Developer',
 'Database_Administrator,Software_Developer,Systems_Administrator,Security_Analyst',
 'Systems_Administrator,Network_Administrator,Security_Analyst',
 'Database_Administrator,Systems_Administrator',
 'Project_manager,Network_Administrator',
 

In [13]:
resume_data[resume_data['Normalized_Occupation']=='']

,ID,Occupations,Resume,Labels,Normalized_Occupation,Preprocessed_Resume
91,C:\Workspace\java\scrape_indeed\dba_part_1\108...,Sr. Database Administrator/Data Analyst;Sr. Da...,Sr. Database Administrator/Data Analyst Sr. <s...,Database_Administrator\nProject_manager,,sr. database administrator/data analyst sr. da...
163,C:\Workspace\java\scrape_indeed\dba_part_1\114...,Used Truck Sales;Benefit Support Specialist / ...,Used Truck Sales Used Truck Sales Used Truck S...,Database_Administrator,,used truck sales used truck sales used truck s...
322,C:\Workspace\java\scrape_indeed\dba_part_1\129...,CLUB COORDINATOR/EVENT COORDINATOR;SUBSTITUTE ...,CLUB COORDINATOR/EVENT COORDINATOR CLUB COORDI...,,,club coordinator/event coordinator club coordi...
357,C:\Workspace\java\scrape_indeed\dba_part_1\132...,City Planner II & Development Review Administr...,City Planner II & Development Review Administr...,,,city planner ii development review administrat...
498,C:\Workspace\java\scrape_indeed\dba_part_1\145...,Business Services Representative - Receptionis...,Business Services Representative - Receptionis...,,,business services representative - receptionis...
...,...,...,...,...,...,...
29530,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web designer / developer;Web designer / UI dev...,"Web designer / developer <span class=""hl"">Web<...",Web_Developer\nSoftware_Developer\nFront_End_D...,,web designer / developer web designer / develo...
29609,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer/PHP Developer/Perl Developer /Py...,Web Developer/PHP Developer/Perl Developer /Py...,Web_Developer\nPython_Developer\nSoftware_Deve...,,web developer/php developer/perl developer /py...
29648,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Sr. Web/UI Developer;Sr. Web/UI Developer;Sr. ...,"Sr. Web/UI Developer Sr. <span class=""hl"">Web<...",Web_Developer\nSoftware_Developer\nFront_End_D...,,sr. web/ui developer sr. web /ui developer sr....
29661,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer/Designer;Product Developer/Autho...,"Web Developer/Designer <span class=""hl"">Web</s...",Web_Developer\nSoftware_Developer,,web developer/designer web developer /designer...


In [12]:
pd.set_option('display.max_colwidth', 1000)
resume_data[['Resume', 'Preprocessed_Resume']]

,Resume,Preprocessed_Resume
0,"Database Administrator <span class=""hl"">Database</span> <span class=""hl"">Administrator</span> Database Administrator - Family Private Care LLC Lawrenceville, GA A self-motivated Production SQL Server Database Administrator who possesses strong analytical and problem solving skills. My experience includes SQL Server 2005, 2008 and 2012, 2014, SSIS, as well as clustering, mirroring, and high availability solutions in OLTP environments. I am proficient in database backup, recovery, performance tuning, maintenance tasks, security, and consolidation. I am confident that I would make a beneficial addition to any company. Over the course of my career thus far, I have designed databases to fit a variety of needs, successfully ensured the security of those databases, problem-solved in order to meet both back-end and front-end needs, installed and tested new versions database management systems, customized and installed applications and meticulously monitored performance for the sm...",database administrator database administrator database administrator - family private care llc lawrenceville ga self-motivated production sql server database administrator possesses strong analytical problem solving skills . experience includes sql server 2005 2008 2012 2014 ssis well clustering mirroring high availability solutions oltp environments . proficient database backup recovery performance tuning maintenance tasks security consolidation . confident would make beneficial addition company . course career thus far designed databases fit variety needs successfully ensured security databases problem-solved order meet back-end front-end needs installed tested new versions database management systems customized installed applications meticulously monitored performance smoothest front-end experience possible . 5 6 years working databases . work experience database administrator family private care llc - roswell ga april 2017 present confirm backups made successfully saved secure ...
1,"Database Administrator <span class=""hl"">Database</span> <span class=""hl"">Administrator</span> sql server database administrator Houston, TX DATABASE ADMINISTRATOR Over five years of experience in database management systems administration, expert at developing and maintaining database, with strong background of working on high-end servers. I would like to establish a fulfilling, proactive career and assume every increasing responsibility in the course of time. Work Experience Database Administrator Intercontinental Registry - Lagos, GU December 2008 to August 2011 Responsibilities Key responsibilities. Planning, Development: Involved in analyzing business requirements, developing and designing data models,  Provided and designed DB tools to assist in the database management, transactions and processing environments.  Assessed and executed implementing of new technologies.  Worked in coordination with company developers and project managers.  Developed training programs ...",database administrator database administrator sql server database administrator houston tx database administrator five years experience database management systems administration expert developing maintaining database strong background working high-end servers . would like establish fulfilling proactive career assume every increasing responsibility course time . work experience database administrator intercontinental registry - lagos gu december 2008 august 2011 responsibilities key responsibilities . planning development involved analyzing business requirements developing designing data models provided designed db tools assist database management transactions processing environments . assessed executed implementing new technologies . worked coordination company developers project managers . developed training programs trained technical support applications personnel utilize on-line databases environment process . technical support monitored performance capaci

In [15]:
resume_data.isnull().sum()

ID                       0
Occupations              0
Resume                   0
Labels                   0
Normalized_Occupation    0
Preprocessed_Resume      0
dtype: int64

In [17]:
resume_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29780 entries, 0 to 29779
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     29780 non-null  object
 1   Occupations            29780 non-null  object
 2   Resume                 29780 non-null  object
 3   Labels                 29780 non-null  object
 4   Normalized_Occupation  29780 non-null  object
 5   Preprocessed_Resume    29780 non-null  object
dtypes: object(6)
memory usage: 1.4+ MB
